(data-proc:additional)=
# Additional information for processed data

This page provide information on some aspects of processed data that may require additional explanation to fully understand the representation and underlying operations.

(data-proc:echo-range)=
## Range of echo samples

The calibration operation in `compute_Sv` generates a new data variable, `echo_range`, which is the physically meaningful range (in meters) of the echo samples.
`echo_range` is compute from the `range_sample` coordinate (which contains 0-based indices of the digitized sample numbers of the received echoes) of the raw data in combination with the `sample_interval` in the `Sonar/Beam_groupX` group and sound speed either stored in the raw data file or provided by the user.
Echopype assumes a constant sound and does not currently support the use a sound speed profile.


The computation for `echo_range` ($r$) is:

$$
r = N \Delta r = N c \tau / 2
$$

where $\Delta r$ is the along-range "length" of each sample, $N$ is the index number in `range_sample`, $\tau$ is the `sample_interval`, and $c$ is sound speed. The factor of 2 is due to the round-trip travel from the transmitter to the scatterer and back to the receiver.

Because `sample_interval` can be different for different channels, the resulting ``echo_range`` arrays can be different across channel even if they have the same number of digitized samples. This is illustrated in the sketch below, in which (**A**) shows the dimensions of the variable `backscatter_r` and (**B**) shows the varying values of `echo_range` that change depending on `sample_interval`. In this example, the `sample_interval` of the first channel is twice of that of the second and the third channel.

![](./images/echo_range.png)

(data-proc:z-variables)=
## Vertical coordinate (_z_-axis) variables

The SONAR-netCDF4 v1 convention defines several variables in the `Platform` group that are referenced to a common Platform Coordinate System (PCS) vertical (_z_-axis) origin, ${PCS}_o$. These variables, when available, enable the calculation of the depth of each echo sample from the water surface. These variables are listed below together with their definitions, with edits when appropriate.

The _z_ axis is positive-down from the platform origin ${PCS}_o$, and all variables except for `vertical_offset` are defined with respect to ${PCS}_o$. All variables are in units meters.

- `water_level`: Distance from the ${PCS}_o$ to the *nominal* (fixed) water level measured along the _z_-axis of the platform coordinate system (positive values are below the origin).
- `vertical_offset`: Time-varying distance from the *nominal* water level to the *actual* water level measured along the _z_-axis of the platform coordinate system (*positive values are when the actual water level is below the nominal water level*). For ships and similar platforms, this is *heave*, but the concept applies equally well to underwater vehicle depth.
- `MRU_offset_z`: Distance along the _z_-axis from ${PCS}_o$ to the motion reference unit (MRU) sensor origin.
- `position_offset_z`: Distance along the _z_-axis from ${PCS}_o$ to the latitude/longitude sensor origin.
- `transducer_offset_z`: Distance along the _z_-axis from ${PCS}_o$ to the sonar transducer.

Using these variables, the depth of the transducer with respect to the actual water level can be calculated as:

```
transducer_depth = transducer_offset_z - water_level - vertical_offset
```

and the depth for a given echo sample is:

```
sample_depth = transducer_depth + echo_range_z
```

where `echo_range_z` is the _z_-axis component (positive down) of `echo_range`.

When the transducer points vertically (up or down), `echo_range_z = echo_range`. When the transducer is mounted with an angle to the vertical axis, the beam direction vector (defined by the variables `beam_direction_x`, `beam_direction_y`, `beam_direction_z` in the `Sonar/Beam_groupX`) should be used together with `echo_range` to calculate `echo_range_z`.

For an echosounder in an underwater vehicle or moored on the seabed, a pressure sensor will provide the reference, time-varying depth. In this case:
- ${PCS}_o$ corresponds to the position of the pressure sensor
- `water_level` as defined by the convention is unnecessary and can be set to 0
- `vertical_offset = -pressure_sensor_depth`

The sketch below shows the relationship between the above vertical coordinate (_z_-axis) variables.

![z_axis_vars](./images/z_axis_vars.png)